# Handle binary payloads from a Managed Online Endpoint

In this example, receiving and sending binary payloads in scoring scripts is demonstrated using the `rawhttp` decorator as well as the `AMLRequest` and `AMLResponse` objects. Without `rawhttp`, the run function is called passed the serialized JSON from the payload. Using `rawhttp`, the run function is instead passed an `AMLRequest` object, which wraps the native Flask request object used internally by the Azure Inference Server. After handling binary payloads, one can either return a JSON-serializable object as usual or use the `AMLResponse` object to have full control over the response, including returning binary payloads. 

## 1. Configure parameters, assets, and clients

### 1.1 Import required libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential
import random, os, requests

### 1.2 Set workspace details

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"

### 1.3 Set variables

In [ ]:
rand = random.randint(0, 10000)

endpoint_name = f"endpt-moe-{rand}"

### 1.4 Download sample data

In [ ]:
url = "https://aka.ms/peacock-pic"
agent = f"Python Requests/{requests.__version__} (https://github.com/Azure/azureml-examples)"
r = requests.get(url, headers={"User-Agent": agent}, allow_redirects=True)
open("binary-payloads/input.jpg", "wb").write(r.content)

### 1.5 Create an MLClient Instance

In [ ]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

## 2. Create endpoint

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

## 3. Create a Binary-to-Binary Deployment

This script receives an image as a binary file and returns a resized image as a binary file. Both scoring scripts use the `rawhttp` decorator to change the argument passed to the run function from JSON to the entire `AMLRequest` object. This script also uses the `AMLResponse` object 

```python
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from PIL import Image
import io 

default_resize = (128, 128)

def init(): 
    pass 

@rawhttp
def run(req : AMLRequest):
    try:
        data = req.files.getlist("file")[0]
    except IndexError:
        return AMLResponse("No file uploaded", status_code=422)
    
    img = Image.open(data.stream)
    img = img.resize(default_resize)

    output = io.BytesIO()
    img.save(output, format="JPEG")
    resp = AMLResponse(message = output.getvalue(), status_code=200)
    resp.mimetype = "image/jpg"

    return resp
```

### 3.1 Create the deployment

In [ ]:
deployment = ManagedOnlineDeployment(
    name="binarypayloads",
    endpoint_name=endpoint_name,
    model=Model(path="binary-payloads"),
    code_configuration=CodeConfiguration(
        code="binary-payloads/code", scoring_script="single-file-to-file-score.py"
    ),
    environment=Environment(
        conda_file="binary-payloads/env.yml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

### 3.2 Create the deployment

In [ ]:
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

### 3.3 Update endpoint traffic

In [ ]:
endpoint.traffic = {"binarypayloads": 100}
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 3.4 Get endpoint details

In [ ]:
scoring_uri = endpoint.scoring_uri
key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key

### 3.5 Test the endpoint

In [ ]:
res = requests.post(
    url=scoring_uri,
    headers={"Authorization": f"Bearer {key}"},
    files=[("file", open("binary-payloads/input.jpg", "rb"))],
)
open("binary-payloads/output.jpg", "wb").write(res.content)

## 5. Create a Binary-to-JSON Deployment

### 5.1 Examine the scoring script
This script accepts multiple image files uploaded as `file[]` and returns the sizes of the images as JSON. Both scoring scripts use the `rawhttp` decorator to change the argument passed to the run function from JSON to the entire `AMLRequest` object. However, unlike the first script this one returns a dictionary rather than an `AMLResponse` object.

```python
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from PIL import Image

def init(): 
    pass 

@rawhttp
def run(req : AMLRequest):
    sizes = [{"filename" : f.filename,
        "size" : Image.open(f.stream).size}
        for f in req.files.getlist("file[]")]

    return {"response" : sizes}
```

### 5.2 Update the deployment

In [ ]:
deployment.code_configuration = CodeConfiguration(
    code=deployment.code_configuration.code,
    scoring_script="code/multi-file-to-json-score.py",
)
deployment = ml_client.online_deployments.begin_create_or_update(deployment).result()

## 6. Test the endpoint

### 6.1 Send a request

In [ ]:
res = requests.post(
    url=scoring_uri,
    headers={"Authorization": f"Bearer {key}"},
    files=[
        ("file[]", open("binary-payloads/input.jpg", "rb")),
        ("file[]", open("binary-payloads/output.jpg", "rb")),
    ],
)

## 7. Delete assets

### 7.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name)